In [ ]:
import numpy as np
    
def masked_3DGP_v2(grid_sz = None,l_scale = None,p_scale = None,mu = None,bin_mask = None,threshold = None,l_weights = None): 
    # [gp_vals] = masked_3DGP_v2(grid_sz, l_scale, p_scale, varargin)
    
    # This function draws from a 3-D GP with mean mu, length-scale l_scale and
# variance scale parameter p_scale. Basically it returns a volume X, where
#             X ~ N(mu, C), where C_{i,j} = p*e^{-(i-j)^2/(2*l^2)}
# The method used is to draw i.i.d. random variables in the frequency
# domain, apply the fft of C_{i,j} point-wise, and ise in ifft to return to
# the spatial domain. This function was modified for speedups and multiple
# spatial scales
    
    # It takes in parameters
#     - grid_sz:  The dimensions of the sample to take (if grid_sz is a
#                 scalar, then the samples in each dimension will be that
#                 value).
#     - l_scale:  The length scale of the GP (i.e. l in the above
#                 definition of C_{i,j})
#     - p_scale:  The covariance scaling of the GP (i.e. p in the above
#                 definition of C_{i,j})
#     - mu:       The mean of the grid size. Can be either a scalar or a
#                 matrix of the same size as X.
#     - bin_mask: OPTIONAL Binary mask to set certain values of the output
#                 to zero
    
    # And returns the output
#     - X:        The sample from the GP, as defined above.
    
    # 2017 - Adam Charles and Alex Song
    
    ###########################################################################
## Check Inputs
    
    if len(varargin) < 7 or len(l_weights)==0:
        l_weights = 1
    
    if len(varargin) < 6 or len(threshold)==0:
        threshold = 1e-10
    
    if len(varargin) < 5 or len(bin_mask)==0:
        bin_mask = 1
    
    if np.size(np.asarray(grid_sz)) == 1:
        grid_sz = grid_sz * np.array([1,1,1])
    
    if np.shape(l_scale[1]) == 1:
        l_scale = l_scale * np.array([1,1,1])
    
    if np.asarray(l_weights).size == 1:
        l_weights = np.ones((np.shape(l_scale[0]),1)) * l_weights
    
    ###########################################################################
## Create Kernel
    
    wmx = np.pi / 2
    grid_x = reshape(single(np.linspace(- wmx,wmx,grid_sz(1))) ** 2,[],1,1)
    grid_y = reshape(single(np.linspace(- wmx,wmx,grid_sz(2))) ** 2,1,[],1)
    grid_z = reshape(single(np.linspace(- wmx,wmx,grid_sz(3))) ** 2,1,1,[])
    gp_vals = np.zeros((grid_sz,'single'))
    for i in np.arange(1,l_scale.shape[1-1]+1).reshape(-1):
        ker_x = np.exp(- grid_x * l_scale(i,1) ** 2)
        ker_y = np.exp(- grid_y * l_scale(i,2) ** 2)
        ker_z = np.exp(- grid_z * l_scale(i,3) ** 2)
        ker_1 = bsxfun(times,bsxfun(times,ker_x,ker_y),ker_z)
        ker_loc = (ker_1 > threshold)
        ker_len = sum(ker_loc)
        if (ker_len < np.prod(grid_sz) / 2):
            sprev = rng
            rng('shuffle','simdTwister')
            TMP = np.random.randn(np.array([ker_len,1]),'single') + 1j * np.random.randn(np.array([ker_len,1]),'single')
            TMP = np.multiply((l_weights(i) * np.sqrt(np.prod(l_scale(i,:)))) * TMP,ker_1(ker_loc))
            np.random.seed(sprev)
            gp_vals[ker_loc] = gp_vals(ker_loc) + TMP
            clear('TMP','ker_loc')
        else:
            clear('ker_loc')
            sprev = rng
            rng('shuffle','simdTwister')
            TMP = np.random.randn(grid_sz,'single') + 1j * np.random.randn(grid_sz,'single')
            TMP = np.multiply((l_weights(i) * np.sqrt(np.prod(l_scale(i,:)))) * TMP,ker_1)
            np.random.seed(sprev)
            gp_vals = gp_vals + TMP
            clear('TMP')
    
    gp_vals = np.sqrt(np.asarray(gp_vals).size) * real(ifftshift(ifftn(ifftshift(gp_vals))))
    
    gp_vals = np.multiply(p_scale * (2 ** 4.5 / np.pi ** 1.5) * bin_mask,gp_vals) / np.sqrt(len(l_weights)) + mu
    
    return gp_vals
